# Resultados DEMO

En primer lugar probaremos la segmentación de una imagen en la demo de la página web.

Slice 30 del dato 29 una FLAIR Y T1 de utrecht.
SLICE 30 del dato  60 una FLAIR y t1 singapore

Vamos a comenzar observando un corte transversal de una FLAIR proviniente de UTRECHT( en concreto la 29)
<div style="display:flex; justify-content: space-around;">
    <img src="imagenSAM/FLAIRUTRECHT.png" alt="GIF 1" style="width:33%;">
    <img src="imagenSAM/MASKUTRECHT.png" alt="GIF 2" style="width:33%;">
</div>

La demo nos permite 3 opciones para segmentar. La primera opción es HOVER AND CLICK. Solo detecta el fondo o el cerebro.

<div style="display:flex; justify-content: space-around;">
<img src="imagenSAM/HOVER&CLICK.gif" alt="GIF 3" style="width:50%;">
</div>

La segunda opción es segment EVERYTHING:Segmenta el cerebro y una parte interior

<div style="display:flex; justify-content: space-around;">
<img src="imagenSAM/Everythting.gif" alt="GIF 3" style="width:50%;">
</div>

Y por último BOX, comenzamos haciendo una caja pequeña que nos permite ampliarla mediante una serie de puntos que hacen que podamos segmentar una de las manchas pero no de una forma tan definida.

<div style="display:flex; justify-content: space-around;">
<img src="imagenSAM/BOX.gif" alt="GIF 3" style="width:50%;">
</div>

Vamos a probar con el correspondiente T1
<div style="display:flex; justify-content: space-around;">
    <img src="imagenSAM/T1UTRECHT.png" alt="GIF 1" style="width:33%;">
    <img src="imagenSAM/MASKUTRECHT.png" alt="GIF 2" style="width:33%;">
</div>

Mostramos el método Box porque los otros dos han producido los mismo resultados. El que mejor ha segmentado ha sido BOX. Gracias a la caja que se crea en un principio, se pueden ir añadiendo puntos para ampliar la máscar y otros para reducirla. Sin embargo, no es correcta totalmente.

<div style="display:flex; justify-content: space-around;">
<img src="imagenSAM/BOXT1.gif" alt="GIF 3" style="width:50%;">
</div>

# Prueba en local 

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import torch

from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

c:\Users\Aina\miniconda3\envs\tfg_aina\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] No se encontró el proceso especificado'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
